# Set Metric-Establish Baseline-Model Selection-Hyperparameter Tuning

This notebook will perform steps 4 and 5:

4. Set Evaluation Metric & Establish Baseline
5. Model Selection & Tune Hyperparameters of the Model

## Imports

In [1]:
# manipulation libraries
import pandas as pd
import numpy as np

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
# to display visuals in the notebook

%config InlineBackend.figure_format='retina'
#to enable high resolution plots

# normalization and random-search and error metric
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# potential machine Learning Models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
import lightgbm as lgb

# to save machine Learning Models
import pickle

## Auxiliary Functions

In [2]:
# functions to use in the notebook
def fit_evaluate_model(model, X_train, y_train, 
                       X_valid, y_valid):
    # function to train a given model
    # return mean squared error of the
    # actuals and predictions
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_valid)
    return mean_squared_error(y_valid, y_predicted)

def convert_features_to_array(features):
    # function to convert feature df
    # to an array
    num_rows = len(features)
    num_cols = len(features.columns)
    
    features_array = (np
                      .array(features)
                      .reshape((num_rows, 
                                num_cols)))

    return features_array

def convert_target_to_array(target):
    # function to convert target df
    # to an array
    target_array = (np
                    .array(target)
                    .reshape((-1, )))
    return target_array

## Load data and convert to array:

In [3]:
X_train = pd.read_csv("../transformed/X_train.csv")
y_train = pd.read_csv("../transformed/y_train.csv")

X_valid = pd.read_csv("../transformed/X_valid.csv")
y_valid = pd.read_csv("../transformed/y_valid.csv")

In [4]:
y_train_array = convert_target_to_array(y_train)
y_valid_array = convert_target_to_array(y_valid)

X_train_array = convert_features_to_array(X_train)
X_valid_array = convert_features_to_array(X_valid)

# Set Evaluation Metric & Establish Baseline

In real-world applications of data science/machine learning, the evaluation metric is set by data scientists in line with the stakeholder’s expectations from the ML model. That is why this is an important step.

## Set Metric

***Mean square error (MSE)*** is the average of sum of squared residuals where a ***residual*** is the a difference between the actual and predicted value of a target variable. In other words, we are going to evaluate our model by looking at the measure of how large our squared errors (residuals) are spread out.

Mean square error is selected as an error metric, because it is interpretable, it is analogous to variance and it also aligns with our selected algorithm's error minimization criteria.  

On the other hand, this error metric is sensitive to extreme values or outliers. Since it takes the square of the differences between the actuals and predictions, in the presence of extreme values and outliers difference grows quadratically.


##  Establish Baseline

Before building moving to the model selection we are going to construct a common-sense baseline with MSE. A ***common-sense baseline*** can be explained as generating a naive guess of the target value by using some expert knowledge or few lines of code.

We are going to select and tune the models that beats this baseline at a significant level. If any of them cannot beat it then machine learning may not be the best approach to solve this problem or whole preprocessing steps needs re-consideration. For our regression problem, a simple common-sense baseline is to predict the variance of the mean of the training set to the validation set. This approach aligns with our evaluation metric MSE as well.

In [5]:
# set baseline as mean of training set's target value
baseline = (np
            .mean(
                y_train_array))
baseline

88.45000745045448

In [6]:
# calculate MSE baseline
mse_baseline = (np
                .mean(
                    np.square(
                        baseline - y_valid_array)))

print("Baseline error is:", round(mse_baseline, 2))

Baseline error is: 9.01


This shows that on average variance between the training points and validation points are 9.01. In other words, sum of squared residuals of average of training points to the validation points is 9.01.

# Select an ML Model Based on the Evaluation Metric

We are going to try one linear algorithm, two distance-based algorithm and three tree-based algorithms:
* Linear regression
* K-nearest neighbors
* Support vector machines
* Random forests regressor
* Extra trees regressor
* Light gradient boostin machines

We are going to train them with the training datasets, and compare their performances by looking at the MSE of the predictions of the validation set.

For now, we will not go into the details of each model, we will observe the run time and the mean squared error reported by each model. We will elaborate on the best model in the last notebook.

## Linear Regresssion

In [7]:
%%time
# linear regression
lr = LinearRegression()
mse_lr=fit_evaluate_model(lr, 
                          X_train_array, 
                          y_train_array, 
                          X_valid_array, 
                          y_valid_array)

print("MSE of linear regression:", mse_lr)

MSE of linear regression: 7.271003914920919
CPU times: user 55 ms, sys: 93 ms, total: 148 ms
Wall time: 26.2 ms


Linear regression slightly decreased the baseline metric, showing that it is not a candidate to be a good predictor.

## Normalize Datasets for KNN and SVM

Distance-based models use the euclidian distance to train the models, thus varying ranges causes distance-based models to generate inaccurate predictions. In order to try different distance-based models, we are going to scale down the features with normalization.

In [8]:
# create scaler
scaler = StandardScaler()

# apply normalization to training set and transform training set
X_train_array_scaled = (scaler
                        .fit_transform(
                            X_train_array, y_train_array))

# transform validation set
X_valid_array_scaled = scaler.transform(X_valid_array)

## K Nearest Neighbors Regressor

In [9]:
%%time
knn = KNeighborsRegressor()
mse_knn = fit_evaluate_model(knn,
                             X_train_array_scaled, 
                             y_train_array, 
                             X_valid_array_scaled, 
                             y_valid_array)

print("MSE of knn regressor:", mse_knn)

MSE of knn regressor: 6.374447921323201
CPU times: user 105 ms, sys: 18.9 ms, total: 124 ms
Wall time: 130 ms


K-nearest regressor performed better than the linear regression however, MSE is still high, showing that this algorithm is not a good predictor.

## Support Vector Machines Regressor

In [10]:
%%time
svm = SVR()
mse_svm = fit_evaluate_model(svm,
                             X_train_array_scaled, 
                             y_train_array, 
                             X_valid_array_scaled, 
                             y_valid_array)

print('MSE of Support Vector Machines:', mse_svm)

MSE of Support Vector Machines: 5.860241966386057
CPU times: user 1.59 s, sys: 10.8 ms, total: 1.6 s
Wall time: 1.6 s


Support vector machines performed better than the k-nearest neighbors at a higher run-time. All in all MSE decreased 35% showing that this algorithm might be a candidate for a building a good predictor.

## Random Forests Regressor

In [11]:
%%time
rf = RandomForestRegressor(random_state=42)
mse_rf = fit_evaluate_model(rf,
                            X_train_array,
                            y_train_array,
                            X_valid_array,
                            y_valid_array)

print("MSE of Random Forests", mse_rf)

MSE of Random Forests 5.405698983941461
CPU times: user 1.91 s, sys: 39.9 ms, total: 1.95 s
Wall time: 1.95 s
